In [50]:
import requests
from bs4 import BeautifulSoup
from io import StringIO
import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def get_realtime_stock(symbol):
    """
    Scrape real-time stock data from Yahoo Finance for a given stock symbol.

    Args:
        symbol (str): The stock symbol (ticker) to fetch data for.

    Returns:
        dict: A dictionary containing stock data, including symbol, company name, price, and change.
    """
    # Set user agent to simulate a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }

    # Construct the URL for the stock symbol
    url = f"https://finance.yahoo.com/quote/{symbol}"

    try:
        # Make an HTTP request to the URL
        r = requests.get(url, headers=headers)
        r.raise_for_status()  # Raise an exception for 4xx or 5xx HTTP status codes

        # Parse the HTML content of the page
        soup = BeautifulSoup(r.text, 'html.parser')

        # Find and extract the real-time stock data
        price_element = soup.find('fin-streamer', class_="Fw(b) Fz(36px) Mb(-4px) D(ib)")
        change_element = soup.find('fin-streamer', class_="Fw(500) Pstart(8px) Fz(24px)")

        # Create a dictionary with the stock data
        stock = {
            'company_name': soup.title.text.split("Stock Price,")[0],  # Extract the company name from the title
            'price': price_element.text if price_element else 'N/A',
            'change': change_element.text if change_element else 'N/A'
        }

        return stock

    except requests.exceptions.RequestException as e:
        print("Error in making the request:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None


In [ ]:
def get_historical_data(symbol):
    """
    Scrape historical data from Yahoo Finance for a given stock symbol.

    Args:
        symbol (str): The stock symbol (ticker) to fetch data for.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing historical stock data.
    """
    # Construct the URL for historical data
    url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'.format(symbol)

    parameters = {
        'range': '7y',
        'interval': '1d',
        'events': 'history',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }

    try:
        # Make an HTTP request to the URL with specified parameters and headers
        response = requests.get(url, params=parameters, headers=headers)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx HTTP status codes

        # Read CSV data into a Pandas DataFrame
        file = StringIO(response.text)
        reader = csv.reader(file)
        data = list(reader)

        # Extract the header row as column names
        header = data[0]

        # Create the Pandas DataFrame excluding the header row
        df = pd.DataFrame(data[1:], columns=header)

        # Convert numeric columns to their appropriate data types (float or int)
        numeric_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)
        # Convert the "Date" column to a date data type
        df['Date'] = pd.to_datetime(df['Date'])

        return df

    except requests.exceptions.RequestException as e:
        print("Error in making the request:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None


In [51]:
# Define a list of stock symbols
stock_symbols = ['AAPL', 'MSFT', 'TSLA', 'GOOG', 'AMZN', 'META']

In [56]:
# Create an empty list to store the results
stock_data_list = []

# Loop through the company symbols and fetch data for each
for symbol in stock_symbols:
    stock_data = get_realtime_stock(symbol)
    if stock_data is not None:
        stock_data_list.append(stock_data)

In [58]:
stock_data_list[0]

{'symbol': 'AAPL',
 'company_name': 'Apple Inc. (AAPL) ',
 'price': '176.65',
 'change': '-0.92'}

In [54]:
# Create an empty dictionary to store the historical data DataFrames
historical_data_dict = {}

# Loop through the list of companies and fetch historical data
for symbol in stock_symbols:
    historical_data = get_historical_data(symbol)
    if historical_data is not None:
        historical_data_dict[symbol] = historical_data

In [59]:
# Access historical data for AAPL
aapl_data = historical_data_dict['AAPL']
aapl_data

# Access historical data for MSFT
# msft_data = historical_data_dict['MSFT']
# msft_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-11-04,27.132500,27.562500,27.027500,27.209999,25.356308,123348000
1,2016-11-07,27.520000,27.627501,27.365000,27.602501,25.722071,130240000
2,2016-11-08,27.577499,27.930000,27.424999,27.764999,25.873495,97016800
3,2016-11-09,27.469999,27.830000,27.012501,27.719999,25.831556,236705600
4,2016-11-10,27.772499,27.772499,26.457500,26.947500,25.111691,228538000
...,...,...,...,...,...,...,...
1756,2023-10-30,169.020004,171.169998,168.869995,170.289993,170.289993,51131000
1757,2023-10-31,169.350006,170.899994,167.899994,170.770004,170.770004,44846000
1758,2023-11-01,171.000000,174.229996,170.119995,173.970001,173.970001,56934900
1759,2023-11-02,175.520004,177.779999,175.460007,177.570007,177.570007,77334800


In [ ]:
# Closing price
plt.figure(figsize=(15, 10))
plt.subplots_adjust(top=1.25, bottom=1.2)

for i, company in enumerate(stock_symbols, 1):
    plt.subplot(2, 3, i)
    
    company['Adj Close'].plot()
    plt.ylabel('Adj Close')
    plt.xlabel(None)
    plt.title(f"Closing Price of {stock_symbols[i - 1]}")
    
plt.tight_layout()